In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import string
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\felic\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
ocr_excel = pd.read_excel('Output/OCR_excel.xlsx')

In [3]:
codes = ocr_excel['Codes']
sn = ocr_excel['S/N']
gender = ocr_excel['Gender']
name = ocr_excel['NAME']

In [4]:
ocr_excel.drop(ocr_excel[((codes == '700') & (sn == 381)) | ((gender.isna()) & (name.isna()))].index, inplace=True)

In [5]:
gender = ocr_excel['Gender']
gender.loc[name.isin(['F', 'M'])] = gender[name.isin(['F', 'M'])].fillna(name)

C:\Users\felic\AppData\Local\Temp\ipykernel_6556\1320310970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender.loc[name.isin(['F', 'M'])] = gender[name.isin(['F', 'M'])].fillna(name)


In [6]:
name = ocr_excel['NAME']
name.replace('F', np.NaN, inplace=True)
name.replace('M', np.NaN, inplace=True)

In [7]:
ocr_excel['Type'].replace('OUPATIENT', 'OUTPATIENT', inplace=True)

In [8]:
for index, row in ocr_excel.iterrows():
    if pd.isna(row['Gender']):
        matching_rows = ocr_excel.loc[codes == row['Codes']]
        for _, matching_row in matching_rows.iterrows():
            if not pd.isna(matching_row['Gender']):
                ocr_excel.at[index, 'DOB'] = matching_row['DOB']
                break

In [9]:
ocr_excel.loc[gender.isnull(), 'Gender'] = \
    ocr_excel.loc[gender.isnull(), 'Codes'].map(
        ocr_excel.dropna(subset=['Gender']).set_index('Codes')['Gender'].to_dict()
    )

In [10]:
def extract_year_from_date(date_string):
    if isinstance(date_string, int):
        return date_string
    date_obj = pd.to_datetime(date_string, errors='coerce')
    if not pd.isnull(date_obj):
        return date_obj.year
    else:
        return None

ocr_excel['DOB'] = ocr_excel['DOB'].apply(extract_year_from_date)

In [11]:
ocr_excel['DOB'] = ocr_excel['DOB'].fillna(ocr_excel.groupby('Codes')['DOB'].transform('last'))

In [12]:
from datetime import datetime

current_year = datetime.now().year
ocr_excel['Age'] = current_year - ocr_excel['DOB']
dob_index = ocr_excel.columns.get_loc('DOB')
ocr_excel.insert(dob_index + 1, 'Age', ocr_excel.pop('Age'))

In [13]:
ocr_excel['Qty'].fillna(ocr_excel['Qty'], inplace = True)

In [14]:
ocr_excel['Subtotal GST'].fillna(ocr_excel['Subtotal']*1.07, inplace = True)

In [15]:
ocr_excel.columns

Index(['S/N', 'Gender', 'DOB', 'Age', 'Consultant', 'Provider', 'Type',
       'Ward Type', 'Invoice No. ', 'Heading', 'Date ', 'Code', 'Description',
       'Qty', 'Total Price', 'Unit Price', 'w GST', 'Subtotal', 'Codes',
       'Hospitals', 'LOG Dx', 'Pt. Ref. No.', 'NAME', 'Subtotal GST',
       'Invoice Period', 'NLP', 'Keywords_Present', 'Keywords_Present2',
       'Keywords_Present3', 'Cancer', 'Year', 'tempmonth', 'tempday', 'Month'],
      dtype='object')

In [16]:
ocr_excel['Heading'].fillna(ocr_excel['Heading'], inplace = True)

In [17]:
#index_to_drop = [17, 25] + list(range(27, 31))
#ocr_excel.drop(ocr_excel.columns[index_to_drop], axis=1, inplace=True)

In [18]:
#ocr_excel.to_excel('OCR_excel (1).xlsx', index=False)

In [19]:
combined_df = ocr_excel

combined_df['Date '] = pd.to_datetime(combined_df['Date '], errors='coerce')
combined_df = combined_df.dropna(subset=['Date '])


In [20]:
combined_df['Date '] = pd.to_datetime(combined_df['Date '])

combined_df['Year'] = combined_df['Date '].dt.year
combined_df['Month'] = combined_df['Date '].dt.month
combined_df['Day'] = combined_df['Date '].dt.day

#combined_df['Month'] = np.where((combined_df['tempmonth'] == 1) & (combined_df['tempday'] == 1), 1, np.maximum(combined_df['tempmonth'], combined_df['tempday']))


In [21]:
combined_df = combined_df[(combined_df['Year'] >= 2016) & (combined_df['Year'] <= 2020)]

In [22]:
def concat_columns(row):
    return ' '.join(str(item) for item in row)

combined_df['NLP'] = combined_df[['Provider', 'Heading', 'Code', 'Description', 'LOG Dx']].apply(concat_columns, axis=1)

descp = combined_df['NLP'].tolist()
unique_desc = list(set(descp))

cat = []

for i in unique_desc:
    s = i.split()
    if s[0] not in cat:
        cat.append(s[0])

In [23]:
get_ma_list = []

for i in unique_desc:
    s = i.split()
    
    for get_str in s:
       if get_str.endswith('oma') and get_str not in get_ma_list:
          get_ma_list.append(get_str)

In [24]:
get_ma_list = list(set([x.lower() for x in get_ma_list]))
get_ma_list = get_ma_list

In [25]:
get_ma_list

['stoma',
 'cholangiocarcinoma',
 '&stoma',
 'osteosarcoma',
 'gluacoma',
 'lymphangioma',
 'pheochromocytoma',
 'schwannoma',
 'astrocytoma',
 'carcinoma',
 'glioblastoma',
 'glaucoma',
 'adenocarcinoma',
 'liposarcoma',
 'neurofibroma',
 'cholanglocarcinoma',
 'myoma',
 'medullablastoma',
 'medulloblastoma',
 'lymphoma',
 'seroma',
 'melanoma',
 'myeloma',
 'retinoblastoma',
 'rhabdomyosarcoma',
 'sarcoma',
 'glioma']

In [26]:
get_excel_file = 'Input/cancer_list.xlsx'
cancer_list = pd.read_excel(get_excel_file)
cancer_word_list = []

for cancer_type in cancer_list['Cancer type'].dropna():
    if cancer_type not in cancer_word_list:
        cancer_word_list.append(cancer_type)

In [27]:
keywords =['cancer','chemo','radiotherapy','endocrine','hormonal','immunotherapy',"immunotherapy", "metastasis", "benign",
         "forceps","viopsy",'Radiosurgery','Oncology', "LABORATORY ", 'Chemotherapy','core','specimen','leukemia' , 
         'leukaemia' , "filgrastim", "tumour","tumors","tumor", "cytology", "chemotherapy-induced", "Ondansetron","lymphangiosarcoma"
            ]

keyword = keywords + cancer_word_list + get_ma_list
#len(keyword)

lowercase = [str(item).lower() if isinstance(item, str) else item for item in keyword]

lowercase_list = lowercase

common_words = {'a','b', 'and', 'or', 'the','for', 'with', 'not', 'by', 'ii','iii','system','is', 'of', 
                'in', 'on', 'at', 'to','1','2','3','4','5','6','7','8','9','12','surgery','1st','2a','3b','complex',
                'c','t3','t4','treatment','type','during','only','core','exam','screening','others','others ','&stoma','stoma','glaucoma','myoma'}

# Input list

# Remove common words from the input list
lowercase_list2 = [word for word in lowercase_list if word not in common_words]

def remove_duplicates(lst):
    seen = set()
    result = []
    for item in lst:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result


unique_list = remove_duplicates(lowercase_list2)

In [30]:
def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation])


porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()


def process_text(text):
    tokens = word_tokenize(text)
    tokens = [remove_punctuation(token.lower()) for token in tokens if token.isalnum()]
    stemmed_tokens = [porter.stem(token) for token in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    return tokens, stemmed_tokens, lemmatized_tokens


def get_wordnet_pos(token):
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


combined_df['Tokens'], combined_df['Stemmed'], combined_df['Lemmatized'] = zip(*combined_df['NLP'].apply(process_text))


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\felic/nltk_data'
    - 'c:\\Users\\felic\\anaconda3\\nltk_data'
    - 'c:\\Users\\felic\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\felic\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\felic\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [ ]:
def concatenate_list(my_list):
    concatenated_string = ""
    for element in my_list:
        concatenated_string += str(element) + " "
    return concatenated_string

def check_keywords(tokens):
    cancerlist = []
    for token in tokens:
        if token in unique_list:
            cancerlist.append(token)
    return concatenate_list(cancerlist)

In [ ]:
"""
def check_keywords(tokens):
    cancerlist = []
    repeatlists = ['cancer','chemo','chemotherapy','biopsy']
    counter = 0
    result_list = []

    for token in tokens:
        if token in unique_list:
            cancerlist.append(token)

    for i in cancer_list:
        for r in repeatlists:
            if r in i:
                counter += 1
        if counter >=1:
            for item in cancerlist:
                if item not in repeatlists:
                    result_list.append(item)

            return result_list
        else:
            return cancer_list
"""

"\ndef check_keywords(tokens):\n    cancerlist = []\n    repeatlists = ['cancer','chemo','chemotherapy','biopsy']\n    counter = 0\n    result_list = []\n\n    for token in tokens:\n        if token in unique_list:\n            cancerlist.append(token)\n\n    for i in cancer_list:\n        for r in repeatlists:\n            if r in i:\n                counter += 1\n        if counter >=1:\n            for item in cancerlist:\n                if item not in repeatlists:\n                    result_list.append(item)\n\n            return result_list\n        else:\n            return cancer_list\n"

In [ ]:
df_cleaned = combined_df

df_cleaned['Keywords_Present'] = df_cleaned['Tokens'].apply(check_keywords)
df_cleaned['Keywords_Present2'] = df_cleaned['Stemmed'].apply(check_keywords)
df_cleaned['Keywords_Present3'] = df_cleaned['Lemmatized'].apply(check_keywords)

def is_empty(lst):
    if len(lst) == 0:
        return 0
    else:
        return 1

df_cleaned['is_non_empty'] = df_cleaned['Keywords_Present'].apply(is_empty)
df_cleaned['is_non_empty2'] = df_cleaned['Keywords_Present2'].apply(is_empty)
df_cleaned['is_non_empty3'] = df_cleaned['Keywords_Present3'].apply(is_empty)

df_cleaned['Cancer'] = df_cleaned['is_non_empty'] + df_cleaned['is_non_empty2'] + df_cleaned['is_non_empty3']

In [ ]:
med = []

for cancer_type in cancer_list['Active ingredient'].dropna():
    med.append(cancer_type)

In [ ]:
df_cleaned['testing'] = df_cleaned['Keywords_Present'] +df_cleaned['Keywords_Present2'] +df_cleaned['Keywords_Present3']

In [ ]:

def checkuniq(lst):
    splits = lst.split(" ")
    uniques = ['cancer','specimen','chemo','tumor']

    if len(splits) >= 1:
        for i in splits:
            if i in uniques:
                split2 = [x for x in splits if x not in uniques]
                if len(split2) >= 1:
                    #print("one")
                    return splits[0]
                else:
                    #print("two")
                    return split2[1]

            else:
                #print("three")
                return splits[0]
    elif len(splits) == 1:
        #print("four")
        return "cancer"
df_cleaned['MainCancer'] = df_cleaned['testing'].apply(checkuniq)


In [ ]:
df_cleaned

,S/N,Gender,DOB,Age,Consultant,Provider,Type,Ward Type,Invoice No.,Heading,...,Month,Day,Tokens,Stemmed,Lemmatized,is_non_empty,is_non_empty2,is_non_empty3,testing,MainCancer
0,1,F,1972,52,NaN,Raffles Hosptial,Inpatient,Normal,2020803751,A&E/SOC SERVICES,...,3,30,"[raffles, hosptial, a, services, 512000, soc, ...","[raffl, hosptial, a, servic, 512000, soc, serv...","[raffle, hosptial, a, service, 512000, soc, se...",0,0,0,,
1,2,F,1972,52,NaN,Raffles Hosptial,Inpatient,Normal,2020803751,ACCOMMODATION,...,3,30,"[raffles, hosptial, accommodation, a080307, si...","[raffl, hosptial, accommod, a080307, singl, ro...","[raffle, hosptial, accommodation, a080307, sin...",0,0,0,,
2,3,F,1972,52,NaN,Raffles Hosptial,Inpatient,Normal,2020803751,ACCOMMODATION,...,4,17,"[raffles, hosptial, accommodation, a080307, si...","[raffl, hosptial, accommod, a080307, singl, ro...","[raffle, hosptial, accommodation, a080307, sin...",0,0,0,,
3,4,F,1972,52,NaN,Raffles Hosptial,Inpatient,Normal,2020803751,CONSULTATION,...,4,3,"[raffles, hosptial, consultation, a070078, war...","[raffl, hosptial, consult, a070078, ward, rmo,...","[raffle, hosptial, consultation, a070078, ward...",0,0,0,,
4,5,F,1972,52,NaN,Raffles Hosptial,Inpatient,Normal,2020803751,CONSULTATION,...,4,14,"[raffles, hosptial, consultation, a010031, sub...","[raffl, hosptial, consult, a010031, sub, consu...","[raffle, hosptial, consultation, a010031, sub,...",0,0,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219590,1815,F,1949,75,NaN,TTSH,Inpatient,Class A1,1219533524G,NON-TREATMENT SERVICES,...,9,20,"[ttsh, services, ms0800, facilitation, adminis...","[ttsh, servic, ms0800, facilit, administr, met...","[ttsh, service, ms0800, facilitation, administ...",1,1,1,cholangiocarcinoma metastasis cholangiocarcino...,cholangiocarcinoma
219591,1816,F,1949,75,NaN,TTSH,Inpatient,HD Class A1,1219533524G,NON-TREATMENT SERVICES,...,9,25,"[ttsh, services, ms0800, facilitation, adminis...","[ttsh, servic, ms0800, facilit, administr, met...","[ttsh, service, ms0800, facilitation, administ...",1,1,1,cholangiocarcinoma metastasis cholangiocarcino...,cholangiocarcinoma
219592,1817,F,1949,75,NaN,TTSH,Inpatient,ICU Class A1,1219533524G,NON-TREATMENT SERVICES,...,9,26,"[ttsh, services, ms0800, facilitation, adminis...","[ttsh, servic, ms0800, facilit, administr, met...","[ttsh, service, ms0800, facilitation, administ...",1,1,1,cholangiocarcinoma metastasis cholangiocarcino...,cholangiocarcinoma
219593,1818,F,1949,75,NaN,TTSH,Inpatient,ICU Class A1,1219533524G,NON-TREATMENT SERVICES,...,9,27,"[ttsh, services, ms0800, facilitation, adminis...","[ttsh, servic, ms0800, facilit, administr, met...","[ttsh, service, ms0800, facilitation, administ...",1,1,1,cholangiocarcinoma metastasis cholangiocarcino...,cholangiocarcinoma


In [ ]:
#hard code exclsuion list
df_cleaned2 = df_cleaned

def removedata(texts):
    exc = ['others']
    if texts in exc:
        return " "
    
#df_cleaned['MainCancer'] = df_cleaned['MainCancer'].apply(removedata)


In [ ]:
#df_cleaned.to_excel('OCR_excel.xlsx', index=False)
def check_keywords2(tokens):
    plist = ["and","511001", "4001000036","4001000038","4001000037",'the','novena','parkway']
    counter = 0 
    for token in tokens:
        if token == "cancer":
            counters = counter - 1
            if tokens[counters] not in plist:
                return tokens[counters]
        counter += 1

df_cleaned['cancer2'] = df_cleaned['Tokens'].apply(check_keywords2)

In [ ]:
def combinecol (cola, colb):
    if cola == "cancer" and colb != " ":
        return colb
    elif cola == " ":
        return colb
    else:
        return cola

In [ ]:
df_cleaned['CancerCat'] = df_cleaned.apply(lambda row: combinecol(row['MainCancer'], row['cancer2']), axis=1)

In [ ]:
df_cleaned['CancerPatient'] = df_cleaned.groupby('Patient')['CancerCat'].transform(lambda x: 1 if x.notna().any() else 0)
df_cleaned

,S/N,Gender,DOB,Age,Consultant,Provider,Type,Ward Type,Invoice No.,Heading,...,Cancer,Year,tempmonth,tempday,Month,Day,MainCancer,CancerCat,Patient,CancerPatient
0,1,F,1972,52,NaN,Raffles Hosptial,Inpatient,Normal,2020803751,A&E/SOC SERVICES,...,0,2020,3,30,3,30,,,Raffles Hosptial-1211,1
1,2,F,1972,52,NaN,Raffles Hosptial,Inpatient,Normal,2020803751,ACCOMMODATION,...,0,2020,3,30,3,30,,,Raffles Hosptial-1211,1
2,3,F,1972,52,NaN,Raffles Hosptial,Inpatient,Normal,2020803751,ACCOMMODATION,...,0,2020,4,17,4,17,,,Raffles Hosptial-1211,1
3,4,F,1972,52,NaN,Raffles Hosptial,Inpatient,Normal,2020803751,CONSULTATION,...,0,2020,4,3,4,3,,,Raffles Hosptial-1211,1
4,5,F,1972,52,NaN,Raffles Hosptial,Inpatient,Normal,2020803751,CONSULTATION,...,0,2020,4,14,4,14,,,Raffles Hosptial-1211,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219590,1815,F,1949,75,NaN,TTSH,Inpatient,Class A1,1219533524G,NON-TREATMENT SERVICES,...,3,2019,9,20,9,20,cholangiocarcinoma,cholangiocarcinoma,TTSH-1148,1
219591,1816,F,1949,75,NaN,TTSH,Inpatient,HD Class A1,1219533524G,NON-TREATMENT SERVICES,...,3,2019,9,25,9,25,cholangiocarcinoma,cholangiocarcinoma,TTSH-1148,1
219592,1817,F,1949,75,NaN,TTSH,Inpatient,ICU Class A1,1219533524G,NON-TREATMENT SERVICES,...,3,2019,9,26,9,26,cholangiocarcinoma,cholangiocarcinoma,TTSH-1148,1
219593,1818,F,1949,75,NaN,TTSH,Inpatient,ICU Class A1,1219533524G,NON-TREATMENT SERVICES,...,3,2019,9,27,9,27,cholangiocarcinoma,cholangiocarcinoma,TTSH-1148,1


In [ ]:

columns_to_drop = ['is_non_empty', 'is_non_empty2','is_non_empty3','Tokens','Stemmed','Lemmatized','testing',
                   'testing','cancer2',"NLP","Keywords_Present",'Keywords_Present2','Keywords_Present3']

df_cleaned = df_cleaned.drop(columns=columns_to_drop)


In [ ]:


def normalise(norm):
    if norm == "leukemia":
        return "leukaemia"

#df_cleaned['Combined'] = df_cleaned['Combined'].apply(normalise)

# Specify the path for the new Excel file
output_excel_path = 'Output/OCR_excel_LY.xlsx'

# Write the combined dataframe to a new Excel file
df_cleaned.to_excel(output_excel_path, index=False)

# Display a message indicating the success
print(f"Combined data has been written to {output_excel_path}")



Combined data has been written to Output/OCR_excel_LY.xlsx


In [ ]:
df_cleaned.head()